# Region level

In [1]:
#hide_input
from mo_covid19.data import *

In [2]:
#hide_input
locations_type = "region"
data = collect_data(type_=locations_type)

data.loc[data.locality_name == 'State of São Paulo', 'locality_name'] = "São Paulo"
data.loc[data.locality_name == 'State of Amazonas', 'locality_name'] = "Amazonas"
data.loc[data.locality_name == 'Lombardy', 'locality_name'] = "Lombardia"

long_regions = data.melt(id_vars=["locality_name", "date"], var_name="category", value_name="value")